#### Claude Code Agent SDK Basics

- https://docs.claude.com/en/docs/agent-sdk/python
- https://www.datacamp.com/tutorial/how-to-use-claude-agent-sdk

In [5]:
import asyncio
import pprint
from claude_agent_sdk import (
    query,
    ClaudeAgentOptions,
    AssistantMessage,
    TextBlock,
    ResultMessage,
)

In [42]:
def print_final_result(msg):
    if isinstance(msg, AssistantMessage):
        for block in msg.content:
            if isinstance(block, TextBlock):
                print(block.text, end="")  # only the outline
    elif isinstance(msg, ResultMessage):
        pass  # let the iterator end naturally

#### Basic query and response message data types 

In [4]:
options = ClaudeAgentOptions(
    model="haiku",
)

input_prompt = "Hi"
print(f"User: {input_prompt}")
print("Example using `query()`")
async for message in query(prompt=input_prompt, options=options):
    pprint.pprint(message)

User: Hi
Example using `query()`
SystemMessage(subtype='init',
              data={'agents': ['Bash',
                               'general-purpose',
                               'statusline-setup',
                               'Explore',
                               'Plan'],
                    'apiKeySource': 'none',
                    'claude_code_version': '2.1.12',
                    'cwd': '/data/home/xiong/dev/Everything_AI/notebooks/Agents_and_Workflow/Claude_Code_Agents_SDK',
                    'mcp_servers': [],
                    'model': 'claude-haiku-4-5-20251001',
                    'output_style': 'default',
                    'permissionMode': 'default',
                    'plugins': [],
                    'session_id': 'aad4b9c4-75f7-4411-b674-39610ecb5426',
                    'skills': [],
                    'slash_commands': ['compact',
                                       'context',
                                       'cost',
                 

In [43]:
async def basic_query():
    options = ClaudeAgentOptions(
        model="sonnet", 
        system_prompt="You are a precise technical copy strategist."
    )

    async for msg in query(prompt=PROMPT, options=options):
        print_final_result(msg)

In [44]:
# if __name__ == "__main__":
#     import asyncio
#     asyncio.run(main())
PROMPT = """Create a crisp markdown outline (H2/H3 bullets) for a 500-word blog post:
Title: Why Sovereign AI Compute Matters in 2026
Audience: CTOs and Heads of AI
Tone: pragmatic, non-hype
Include: 3 buyer pains, 3 evaluation criteria, 1 closing CTA
"""
await basic_query()

# Why Sovereign AI Compute Matters in 2026

## H2: The Control Problem We're Not Talking About
- Your AI roadmap relies on infrastructure you don't control
- Three painful realities CTOs face today:
  - **Regulatory whiplash**: EU AI Act, data localization mandates, and changing compliance requirements make cross-border GPU dependency a liability
  - **Capacity rationing**: Hyperscaler queues and allocation caps mean your Q2 model training waits on their priorities, not yours
  - **Vendor lock-in at the silicon level**: Switching costs aren't just about code—they're about who owns the physical compute layer your competitive advantage runs on

## H2: What Sovereignty Actually Means (Beyond the Buzzword)
- Not about nationalism—about operational control and risk management
- Geographic data residency + hardware you can audit + contracts you can enforce
- The difference between "cloud region" sovereignty (still someone else's hardware) and true compute sovereignty

## H2: Three Questions 

#### Calude code agent sdk options 

Claude Code's system prompt includes:

- Tool usage instructions and available tools
- Code style and formatting guidelines
- Response tone and verbosity settings
- Security and safety instructions
- Context about the current working directory and environment
- https://platform.claude.com/docs/en/agent-sdk/modifying-system-prompts

In [ ]:
async def advanced_query(prompt):
    options = ClaudeAgentOptions(
        model="sonnet", 
        permission_mode='bypassPermissions', ## dangerous  ##"default"
        cwd="/data/home/xiong/dev/Everything_AI/notebooks/Agents_and_Workflow/Claude_Code_Agents_SDK",
        setting_sources=["project"], # Required to load CLAUDE.md from project
        settings='{"outputStyle": "Personal Assistant"}', ## this need to be the name of the style not file name ; it is loading from ~/.claude/output-styles/personal-assistant.md
        # system_prompt="You are a pirate. You must respond like a pirate.",
        # add_dirs=["."], # allow access to other directories
        # system_prompt="You are a precise technical copy strategist.",
        # system_prompt={
        #     "type": "preset",
        #     "preset": "claude_code"  # Use Claude Code's system prompt
        # },
        # system_prompt={  ## this will use claude system prompt and then modify it by append specific instructions 
        #     "type": "preset",
        #     "preset": "claude_code",
        #     "append": """
        #     Your name is XYZ
        #     """
        # }
        # hooks={
        #     "on_response": lambda response: print(f"Response: {response}")
        # }
    )

    async for msg in query(prompt=prompt, options=options):
        print_final_result(msg)

In [46]:
PROMPT = """what is your name?""" 
await advanced_query(prompt=PROMPT) ## a "hello_world.py" file will be created in the cwd

My name is **XXYYZZ**, and I'm your personal assistant! I'm here to help you maximize your potential and achieve your goals by providing you with the information and tools you need to succeed.

How can I help you today?

#### ClaudeSDKClient

- use client can manage chat history automitically 
- can support customized tools

In [32]:
from claude_agent_sdk import ClaudeSDKClient, AssistantMessage, TextBlock, ResultMessage

In [11]:
async def chat_with_history():
    async with ClaudeSDKClient() as client:
        # First question
        await client.query("What's the capital of France?")

        # Process response
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")

        # Follow-up question - Claude remembers the previous context
        await client.query("What's the population of that city?")

        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")

        # Another follow-up - still in the same conversation
        await client.query("What are some famous landmarks there?")

        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")



In [12]:
await chat_with_history()

Claude: The capital of France is Paris.
Claude: Paris has a population of approximately 2.1 million people in the city proper (intra-muros). The greater Paris metropolitan area (Île-de-France) has around 12-13 million inhabitants, making it one of the largest urban areas in Europe.
Claude: Paris has many famous landmarks, including:

- **Eiffel Tower** - The iconic iron lattice tower built in 1889
- **Notre-Dame Cathedral** - Gothic cathedral on Île de la Cité (currently under restoration after the 2019 fire)
- **Louvre Museum** - World's largest art museum, home to the Mona Lisa
- **Arc de Triomphe** - Monument at the western end of the Champs-Élysées
- **Sacré-Cœur Basilica** - White-domed church atop Montmartre hill
- **Musée d'Orsay** - Art museum in a former railway station
- **Panthéon** - Mausoleum containing remains of distinguished French citizens
- **Palace of Versailles** - Royal château just outside the city
- **Champs-Élysées** - Famous avenue lined with shops and cafés
- 

#### Tools 
- A decorator function that wraps the tool implementation and returns an SdkMcpTool instance.

In [30]:
from typing import Any
from claude_agent_sdk import (
    query,
    ClaudeAgentOptions,
    tool,
    create_sdk_mcp_server,
    AssistantMessage,
    TextBlock,
    ToolUseBlock,
    ToolResultBlock,
    ResultMessage,
)
import json

In [ ]:
# def tool(
#     name: str,
#     description: str,
#     input_schema: type | dict[str, Any]
# ) -> Callable[[Callable[[Any], Awaitable[dict[str, Any]]]], SdkMcpTool[Any]]
    
@tool(name="add", description="Add two numbers", input_schema={"a": float, "b": float})
async def add(args):
    return {
        "content": [{
            "type": "text",
            "text": f"Sum: {args['a'] + args['b']}"
        }]
    }

@tool(name="multiply", description="Multiply two numbers", input_schema={"a": float, "b": float})
async def multiply(args):
    return {
        "content": [{
            "type": "text",
            "text": f"Product: {args['a'] * args['b']}"
        }]
    }
    


In [ ]:
calculator = create_sdk_mcp_server(
    name="calculator",
    version="2.0.0",
    tools=[add, multiply]  # Pass decorated functions
)

In [27]:
tools_options = ClaudeAgentOptions(
    mcp_servers={"calc": calculator},
    allowed_tools=["mcp__calc__add", "mcp__calc__multiply"],
    permission_mode='bypassPermissions', ## dangerous 
    cwd="/data/home/xiong/dev/Everything_AI/notebooks/Agents_and_Workflow/Claude_Code_Agents_SDK",
    system_prompt="You are a help assistant.",
    model="sonnet"
)

In [30]:
async def chat_with_tools():
    async with ClaudeSDKClient(options=tools_options) as client:
        # First question
        await client.query("What's the 1+1?")

        # Process response
        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")
                    elif isinstance(block, ToolUseBlock):
                        print(f"Tool Use: {block.name} with args: {block.input}")
                    elif isinstance(block, ToolResultBlock):
                        print(f"Tool Result: {block.content}")

        # Follow-up question - Claude remembers the previous context
        await client.query("Then multiply the result by 2")

        async for message in client.receive_response():
            if isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, TextBlock):
                        print(f"Claude: {block.text}")
                    elif isinstance(block, ToolUseBlock):
                        print(f"Tool Use: {block.name} with args: {block.input}")
                    elif isinstance(block, ToolResultBlock):
                        print(f"Tool Result: {block.content}")

In [31]:
await chat_with_tools()

Tool Use: mcp__calc__add with args: {'a': 1, 'b': 1}
Claude: 1 + 1 = **2**
Tool Use: mcp__calc__multiply with args: {'a': 2, 'b': 2}
Claude: 2 × 2 = **4**


### MCPs
- https://platform.claude.com/docs/en/agent-sdk/mcp

In [53]:
import os
from dotenv import load_dotenv
load_dotenv()

True

- add rube service to connect to a wide range of apps with 1 auth

In [57]:
async def test_mcp():
    options = ClaudeAgentOptions(
        mcp_servers={
            "claude-code-docs": {
                "type": "http",
                "url": "https://code.claude.com/docs/mcp"
            },
            "rube": {
                "type": "http",
                "url": "https://rube.app/mcp/http",
                "headers": {
                    "Authorization": f"Bearer {os.environ['RUBE_token']}"
                }
            }
        },
        allowed_tools=["mcp__claude-code-docs__*", "mcp__rube__*"] ## use all tools in the mcp server 
    )

    # async for message in query(prompt="Use the docs MCP server to explain what hooks are in Claude Code", options=options):
    #     if isinstance(message, ResultMessage) and message.subtype == "success":
    #         print(message.result)
    async for message in query(prompt="read my most recent email", options=options):
        if isinstance(message, ResultMessage) and message.subtype == "success":
            print(message.result)
## test it out 
await test_mcp()

I don't have the ability to read your emails. I don't have access to email services, inboxes, or any personal accounts.

To help you with email-related tasks, you would need to:

1. **Copy and paste** the email content directly into this chat
2. **Use an email client** or webmail interface to access your messages
3. **Set up an MCP server** that connects to your email service if you want me to have that capability

Is there something else I can help you with, or would you like to paste the email content here for me to review?


### Sub Agents
- https://platform.claude.com/docs/en/agent-sdk/subagents
- Subagents are invoked via the Task tool. To detect when a subagent is invoked, check for tool_use blocks with name: "Task". Messages from within a subagent's context include a parent_tool_use_id field.

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions, AgentDefinition

True

In [39]:
async def sub_agent_test():
    async for message in query(
        prompt="Use the code-reviewer agent to review this codebase",
        options=ClaudeAgentOptions(
            allowed_tools=["Read", "Glob", "Grep", "Task"],
            agents={  ## you can define a agent in run time 
                "code-reviewer": AgentDefinition( ## this will override the agent defiend in .claude/agents folder
                    description="Expert code reviewer.", 
                    prompt="Analyze code quality and suggest improvements.",
                    tools=["Read", "Glob", "Grep"]
                )
                ## it will also loa dagents defiend in .claude/agents folder
            }
        )
    ):
        # Check for subagent invocation in message content
        if hasattr(message, 'content') and message.content:
            for block in message.content:
                if getattr(block, 'type', None) == 'tool_use' and block.name == 'Task': ## interesting, only tasks are dispatched to subagents
                    print(f"Subagent invoked: {block.input.get('subagent_type')}")

        # Check if this message is from within a subagent's context
        if hasattr(message, 'parent_tool_use_id') and message.parent_tool_use_id:
            print("  (running inside subagent)")

        if hasattr(message, "result"):
            print(message.result)
##
await sub_agent_test()

  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
  (running inside subagent)
The code review is complete. Here's a summary of the findings:

## Code Review Summary

### Critical Issues
1. **Boolean argument parsing bug** in `cli_tools.py:35,38` - Using `default="Fa

In [58]:
options = ClaudeAgentOptions(
    model="sonnet", 
    permission_mode='bypassPermissions', ## dangerous  ##"default"
    cwd="/data/home/xiong/dev/Everything_AI/notebooks/Agents_and_Workflow/Claude_Code_Agents_SDK",
    setting_sources=["project"], # Required to load CLAUDE.md from project
    allowed_tools=["Skill"],
    settings='{"outputStyle": "Personal Assistant"}',
)

async for message in query(
    prompt="What Skills are available?",
    options=options
):
    print_final_result(message)

Based on the Claude Code configuration in this repository, there is **one skill** currently available:

## Available Skills

### **data-analysis** 
- **Location**: `.claude/skills/data-analysis/SKILL.md`
- **Invocation**: `/data-analysis`
- **Purpose**: Comprehensive data analysis skill that provides:
  - Statistical analysis
  - Pattern recognition
  - Visualization
  - Insight generation for datasets

This skill can be invoked by typing `/data-analysis` in the chat, and it specializes in working with datasets like CSV files, Excel files, JSON data, and performing various analytical tasks.

---

**Note:** As XXYYZZ, I can also delegate tasks to specialized subagents (youtube-analyst, researcher, documentation-writer), but those are agents rather than skills. The data-analysis skill is the only user-invocable skill currently configured in this project.